In [ ]:
# !pip install torch torchvision --force-reinstall

from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/drive/MyDrive/SMAI_Project/images_train /content/
!cp -r /content/drive/MyDrive/SMAI_Project/images_val /content/
!cp -r /content/drive/MyDrive/SMAI_Project/images_test /content/
!cp -r /content/drive/MyDrive/SMAI_Project/labels_train.csv /content/
!cp -r /content/drive/MyDrive/SMAI_Project/labels_val.csv /content/

In [1]:
val_csv_path = 'labels_val.csv'
train_csv_path = '/content/labels_train.csv'
train_img_dir = '/content/images_train'
val_img_dir = '/content/images_val'
lat_long_output_csv = '/content/lat-long.csv'
region_output_csv = '/content/region.csv'
angle_output_csv = '/content/angle.csv'
test_img_dir = '/content/images_test'

In [2]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models
import timm

ModuleNotFoundError: No module named 'torch'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"💻 Using device: {device}")

In [ ]:
train_df = pd.read_csv(train_csv_path)
val_df = pd.read_csv(val_csv_path)
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
])


val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])
test_transform = val_transform

In [ ]:
class CompassDataset(Dataset):
    def __init__(self, img_dir, csv_path, transform=None):
        self.df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform
        angles_rad = np.deg2rad(self.df['angle'].astype(float).values)
        self.sin_targets = np.sin(angles_rad)
        self.cos_targets = np.cos(angles_rad)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        target = torch.tensor([self.sin_targets[idx], self.cos_targets[idx]], dtype=torch.float32)
        return image, target

In [ ]:
train_dataset = CompassDataset(train_img_dir, train_csv_path, transform=train_transform)
val_dataset = CompassDataset(val_img_dir, val_csv_path, transform=val_transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)